In [70]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pylab import hist, show
import scipy
import zipfile
from math import log


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

#### $Win = \beta_{0} + \beta_{1}MeanC + \beta_{2}MeanW + \beta_{3}MeanD + e_{s}$

- A balanced roster will have one player ranked in each centre position (C1, C2, C3, C4), two wingers ranked on every line (LW1, RW1, LW2,RW2, etc) and two defensemen ranked in all three defensive pairings. 
- The ranking of a balanced roster is 2.5 [(1+2+3+4)/4] centres, 2.5 [(1+1+2+2+3+3+4+4)/8] for wingers and 2 [(1+1+2+2+3+3)/6] for defensemen.

- Since players are ranked from 1 to 4 for forwards and 1 to 3 for defensemen, 1 being the highest ranked, a team is considered to have an above average roster when the **mean of each forward position is smaller than 2.5 and the mean of defensive pairings is smaller than 2**. 

### games with 4 centers, 8 wingers, 6 defensemen

In [71]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/data/2010_2017_4c_8w_6d_game_team.csv')
#da = pd.readcsv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/data/2010_2017_games_with_4c_8w_6d_1g.csv')
da = da.drop('Unnamed: 0', axis=1)

In [72]:
da.shape

(15794, 17)

In [73]:
da['playercount'] = da.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName',])['PlayerName'].transform('count')
da['rosterposition'] = da.groupby(['Season', 'GameNumber', 'TeamCode', 'Position', 'Rank'])['playercount'].transform('sum')

da.head()

,Season,GameNumber,TeamCode,PlayerName,Position,Rank,GF,GA,GD,WinTeam,LossTeam,RosterCount,PositionCount,CCount,WCount,DCount,GCount,playercount,rosterposition
0,2010,20041,VAN,RYAN KESLER,C,1.0,3,4,-1,ANA,VAN,19,4,4.0,8.0,6.0,1.0,1,2
1,2010,20041,VAN,MANNY MALHOTRA,C,3.0,3,4,-1,ANA,VAN,19,4,4.0,8.0,6.0,1.0,1,1
2,2010,20041,VAN,JANNIK HANSEN,W,2.0,3,4,-1,ANA,VAN,19,8,4.0,8.0,6.0,1.0,1,2
3,2010,20041,VAN,HENRIK SEDIN,C,1.0,3,4,-1,ANA,VAN,19,4,4.0,8.0,6.0,1.0,1,2
4,2010,20041,VAN,RICK RYPIEN,C,4.0,3,4,-1,ANA,VAN,19,4,4.0,8.0,6.0,1.0,1,1


#### pivot table

- the next step is to group players by gamenumber, teamcode, position and rank, to display the quality of players each team has per position. **Pivot table** by player position and rank using roster position values. Game number and team are the indexes. We want to join the levels to generate columns by roster position and rank. 

In [74]:
da = pd.pivot_table(da, index=['Season', 'GameNumber', 'WinTeam', 'LossTeam', 'GF', 'GA', 'GD', 'TeamCode', 'RosterCount', 'CCount', 'WCount', 'DCount', 'GCount'], columns=['Position', 'Rank'], values=['rosterposition'])
da = da.reset_index()
da.columns = ['_'.join(str(s).strip() for s in col if s) for col in da.columns]
da.reset_index()
da = da.fillna(0)
da = da.rename(columns={'rosterposition_C_1.0': 'C1', 'rosterposition_C_2.0': 'C2', 'rosterposition_C_3.0': 'C3', 'rosterposition_C_4.0': 'C4', 'rosterposition_W_1.0': 'W1', 'rosterposition_W_2.0': 'W2', 'rosterposition_W_3.0': 'W3', 'rosterposition_W_4.0': 'W4', 'rosterposition_D_1.0': 'D1', 'rosterposition_D_2.0': 'D2', 'rosterposition_D_3.0': 'D3', 'rosterposition_G_1.0': 'G1', 'rosterposition_G_2.0': 'G2', 'rosterposition_G_3.0': 'G3' })
da = da[['Season', 'GameNumber', 'TeamCode', 'GF', 'GA', 'GD', 'WinTeam', 'LossTeam', 'RosterCount', 'CCount', 'WCount', 'DCount', 'GCount', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'G1', 'G2', 'G3', 'W1', 'W2', 'W3', 'W4']]
da.head(10)

,Season,GameNumber,TeamCode,GF,GA,GD,WinTeam,LossTeam,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4
0,2010,20041,VAN,3,4,-1,ANA,VAN,19,4.0,8.0,6.0,1.0,2.0,0.0,1.0,1.0,2.0,3.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,2.0
1,2010,20041,ANA,4,3,1,ANA,VAN,19,4.0,8.0,6.0,1.0,1.0,1.0,2.0,0.0,2.0,0.0,4.0,1.0,0.0,0.0,3.0,1.0,3.0,1.0
2,2010,20061,MIN,2,3,-1,CBJ,MIN,19,4.0,8.0,6.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,3.0,0.0,1.0,0.0,1.0,2.0,3.0,2.0
3,2010,20061,CBJ,3,2,1,CBJ,MIN,19,4.0,8.0,6.0,1.0,1.0,2.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0,1.0,1.0,4.0,3.0,0.0
4,2010,20076,VAN,2,6,-4,MIN,VAN,20,4.0,8.0,6.0,2.0,2.0,0.0,1.0,1.0,2.0,1.0,3.0,2.0,0.0,0.0,3.0,2.0,1.0,2.0
5,2010,20076,MIN,6,2,4,MIN,VAN,19,4.0,8.0,6.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,3.0,0.0,1.0,0.0,1.0,2.0,3.0,2.0
6,2010,20084,PHI,2,3,-1,ANA,PHI,19,4.0,8.0,6.0,1.0,2.0,0.0,2.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,5.0,2.0,0.0,1.0
7,2010,20084,ANA,3,2,1,ANA,PHI,19,4.0,8.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,0.0,1.0,3.0,1.0,3.0,1.0
8,2010,20101,TOR,2,5,-3,PHI,TOR,19,4.0,8.0,6.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,3.0,2.0,1.0
9,2010,20101,PHI,5,2,3,PHI,TOR,19,4.0,8.0,6.0,1.0,2.0,0.0,2.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,5.0,1.0,0.0,2.0


- Assign a value of 1 to the team that won the game and a value of 0 to the team that loss. Compute the mean by position per team for each game.

In [75]:
da['Win'] = da.apply(lambda x: 1 if x['WinTeam']== x['TeamCode'] else 0, 1)
da['MeanC'] = ((da['C1']*1) + (da['C2']*2) + (da['C3']*3) + (da['C4'] *4))/da['CCount']
da['MeanW'] = ((da['W1']*1) + (da['W2']*2) + (da['W3']*3) + (da['W4'] *4))/da['WCount']
da['MeanD'] = ((da['D1']*1) + (da['D2']*2) + (da['D3']*3))/da['DCount']
da['MeanG'] = ((da['G1']*1) + (da['G2']*2) + (da['G3']*3))/da['GCount']
da.sort_values(['GameNumber'], ascending=[True], inplace=True)
da.head()

,Season,GameNumber,TeamCode,GF,GA,GD,WinTeam,LossTeam,RosterCount,CCount,WCount,DCount,GCount,C1,C2,C3,C4,D1,D2,D3,G1,G2,G3,W1,W2,W3,W4,Win,MeanC,MeanW,MeanD,MeanG
726,2017,20003,CGY,0,3,-3,EDM,CGY,19,4.0,8.0,6.0,1.0,1.0,0.0,3.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,1.0,0,2.50,2.250,2.000000,1.0
727,2017,20003,EDM,3,0,3,EDM,CGY,19,4.0,8.0,6.0,1.0,3.0,1.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,3.0,4.0,1.0,1,1.25,2.750,2.333333,2.0
640,2016,20007,WSH,4,5,-1,PIT,WSH,19,4.0,8.0,6.0,1.0,2.0,2.0,0.0,0.0,4.0,2.0,0.0,1.0,0.0,0.0,5.0,1.0,1.0,1.0,0,1.50,1.750,1.333333,1.0
641,2016,20007,PIT,5,4,1,PIT,WSH,19,4.0,8.0,6.0,1.0,2.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,1.0,1,1.50,1.875,1.666667,2.0
370,2013,20008,N.J,0,3,-3,PIT,N.J,19,4.0,8.0,6.0,1.0,1.0,2.0,1.0,0.0,5.0,1.0,0.0,0.0,1.0,0.0,2.0,4.0,2.0,0.0,0,2.00,2.000,1.166667,2.0


In [76]:
me = da.copy()
me = me[me['Season'] == 2010]
me.shape

(156, 32)

In [77]:
da.shape

(828, 32)

In [78]:
da['Season'].value_counts()

2010    156
2011    154
2017    102
2014    102
2013    100
2016     86
2015     68
2012     60
Name: Season, dtype: int64

In [79]:
df = da.groupby(['Win'])['MeanC',  'MeanW',  'MeanD'].mean()
df =  df.T
df['bf']  =  df[1]/df[0]
df

Win,0,1,bf
MeanC,2.146739,2.036836,0.948805
MeanW,2.171498,2.063104,0.950083
MeanD,1.955314,1.845411,0.943792


### summary analysis

In [80]:
da.groupby(['Win'])['MeanC', 'MeanW', 'MeanD'].describe()

MeanC       MeanW       MeanD
Win                                          
0   count  414.000000  414.000000  414.000000
    mean     2.146739    2.171498    1.955314
    std      0.446328    0.347439    0.356277
    min      1.000000    1.000000    1.166667
    25%      1.750000    2.000000    1.666667
    50%      2.250000    2.125000    2.000000
    75%      2.500000    2.375000    2.166667
    max      3.500000    3.125000    2.833333
1   count  414.000000  414.000000  414.000000
    mean     2.036836    2.063104    1.845411
    std      0.428977    0.344419    0.335328
    min      1.000000    1.000000    1.166667
    25%      1.750000    1.875000    1.666667
    50%      2.000000    2.000000    1.833333
    75%      2.250000    2.250000    2.000000
    max      3.500000    3.000000    2.833333

### model estimation

- regress **team win percent** on the mean of players by position for games with 4 centers, 8 wingers and 6 defensemen.

In [81]:
da['meanc'] = 2.5 - da['MeanC']
da['meanw'] = (2.5 - da['MeanW'])*2
da['meand'] = (2 - da['MeanD'])*2
da['meang'] = 2 - da['MeanG']

In [185]:
y1 = da['Win'] 
y2 = da['GF']
y3 = da['GA']
y4 = da['GD']

X1 = sm.add_constant(da[['meanc', 'meanw', 'meand']] )

m1 = sm.OLS(y1, X1).fit()
m2 = sm.OLS(y2, X1).fit()
m3 = sm.OLS(y3, X1).fit()
m4 = sm.OLS(y4, X1).fit()

m1.summary()
#m2.summary()
#m3.summary()
#m4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Win   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     11.59
Date:                Wed, 25 Jul 2018   Prob (F-statistic):           1.91e-07
Time:                        08:01:43   Log-Likelihood:                -583.85
No. Observations:                 828   AIC:                             1176.
Df Residuals:                     824   BIC:                             1195.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.3954      0.029     13.861      0.000         0.339     0.451
meanc          0.0810      0.041      1.971      0.049         0.000     0.162
meanw          0.0753      0.026      2.870      0.004         0.024     0.127
meand          0.0694      0.027      2.567      0.010         0.016     0.122
==============================================================================
Omnibus:                        0.000   Durbin-Watson:                   2.875
Prob(Omnibus):                  1.000   Jarque-Bera (JB):              116.377
Skew:                          -0.000   Prob(JB):                     5.36e-26
Kurtosis:                       1.163   Cond. No.                         3.84
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- the value of an elite center, winger and defenseman

In [186]:
st = da.copy()
st['lnPC1'] = np.log((st['C1']/4)*100 +1)
st['lnPW1'] = np.log((st['W1']/8)*100 +1)
st['lnPD1'] = np.log((st['D1']/6)*100 +1)
st['lnPC2'] = np.log((st['C2']/4)*100 +1)
st['lnPW2'] = np.log((st['W2']/8)*100 +1)
st['lnPD2'] = np.log((st['D2']/6)*100 +1)
st['lnPC3'] = np.log((st['C3']/4)*100 +1)
st['lnPW3'] = np.log((st['W3']/8)*100 +1)
st['lnPD3'] = np.log((st['D3']/6)*100 +1)
st['lnPC4'] = np.log((st['C4']/4)*100 +1)
st['lnPW4'] = np.log((st['W4']/8)*100 +1)


In [190]:
z1 = st['Win'] 
z2 = st['GF']
z3 = st['GA']
z4 = st['GD']

X2 = sm.add_constant(st[['lnPC1', 'lnPW1', 'lnPD1']])
#X3 = sm.add_constant(st[['lnPC1', 'lnPW1', 'lnPD1', 'lnPC2', 'lnPW2', 'lnPD2', 'lnPC3', 'lnPW3', 'lnPD3', 'lnPC4', 'lnPW4' ]] )
 

n1 = sm.OLS(z1, X2).fit()
n2 = sm.OLS(z2, X2).fit()
n3 = sm.OLS(z3, X2).fit()
n4 = sm.OLS(z4, X2).fit()

#n1.summary()
#n2.summary()
#n3.summary()
n4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     GD   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     9.763
Date:                Wed, 25 Jul 2018   Prob (F-statistic):           2.47e-06
Time:                        08:29:44   Log-Likelihood:                -1916.3
No. Observations:                 828   AIC:                             3841.
Df Residuals:                     824   BIC:                             3859.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -1.7004      0.356     -4.775      0.000        -2.399    -1.001
lnPC1          0.1633      0.058      2.834      0.005         0.050     0.276
lnPW1          0.2066      0.096      2.159      0.031         0.019     0.394
lnPD1          0.1804      0.077      2.354      0.019         0.030     0.331
==============================================================================
Omnibus:                        6.418   Durbin-Watson:                   2.846
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                4.599
Skew:                           0.000   Prob(JB):                        0.100
Kurtosis:                       2.635   Cond. No.                         23.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### calculate  and inspect Variance Inflation Factor (VIF)

- Not correlated: $VIF=1$
- Moderately correlated: **$1<VIF<5$** or at a more conservative level of **$1<VIF <2.5$**
- Highly correlated: **$VIF>=5$** or at a more conservative level **$VIF>=5$**

#### $Win = \beta_{0} + \beta_{1}MeanC + \beta_{2}MeanW + \beta_{3}MeanD + e_{s}$

In [88]:
# For each X1, calculate VIF and save in dataframe
vif1 = pd.DataFrame()
vif1['VIF Factor'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif1['features'] = X1.columns
vif1.round(3)

,VIF Factor,features
0,2.796,const
1,1.127,meanc
2,1.158,meanw
3,1.229,meand


In [89]:
# For each X2, calculate VIF and save in dataframe
vif2 = pd.DataFrame()
vif2['VIF Factor'] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif2['features'] = X2.columns
vif2.round(3)

,VIF Factor,features
0,17.429,const
1,1.119,lnPC1
2,1.121,lnPW1
3,1.066,lnPD1


In [192]:
dt = da.copy()
dt['c0'] = dt.apply(lambda x: 1 if x['C1'] == 0 else 0, axis=1)
dt['c1'] = dt.apply(lambda x: 1 if x['C1'] == 1 else 0, axis=1)
dt['c2'] = dt.apply(lambda x: 1 if x['C1'] == 2 else 0, axis=1)
dt['c3'] = dt.apply(lambda x: 1 if x['C1'] == 3 else 0, axis=1)
dt['c4'] = dt.apply(lambda x: 1 if x['C1'] == 4 else 0, axis=1)

In [193]:
dt['w0'] = dt.apply(lambda x: 1 if x['W1'] == 0 else 0, axis=1)
dt['w1'] = dt.apply(lambda x: 1 if x['W1'] == 1 else 0, axis=1)
dt['w2'] = dt.apply(lambda x: 1 if x['W1'] == 2 else 0, axis=1)
dt['w3'] = dt.apply(lambda x: 1 if x['W1'] == 3 else 0, axis=1)
dt['w4'] = dt.apply(lambda x: 1 if x['W1'] == 4 else 0, axis=1)
dt['w5'] = dt.apply(lambda x: 1 if x['W1'] == 5 else 0, axis=1)
dt['w6'] = dt.apply(lambda x: 1 if x['W1'] == 6 else 0, axis=1)
dt['w7'] = dt.apply(lambda x: 1 if x['W1'] == 7 else 0, axis=1)
dt['w8'] = dt.apply(lambda x: 1 if x['W1'] == 8 else 0, axis=1)

In [194]:
dt['d0'] = dt.apply(lambda x: 1 if x['D1'] == 0 else 0, axis=1)
dt['d1'] = dt.apply(lambda x: 1 if x['D1'] == 1 else 0, axis=1)
dt['d2'] = dt.apply(lambda x: 1 if x['D1'] == 2 else 0, axis=1)
dt['d3'] = dt.apply(lambda x: 1 if x['D1'] == 3 else 0, axis=1)
dt['d4'] = dt.apply(lambda x: 1 if x['D1'] == 4 else 0, axis=1)
dt['d5'] = dt.apply(lambda x: 1 if x['D1'] == 5 else 0, axis=1)
dt['d6'] = dt.apply(lambda x: 1 if x['D1'] == 6 else 0, axis=1)

In [213]:
v1 = dt['Win'] 
v2 = dt['GD']
v3 = dt['GF']
v4 = dt['GA']


X3 = dt[['c0', 'c1', 'c2', 'c3', 'c4']]
X4 = dt[['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8']]
X5 = dt[['d0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6']]

#X6 = dt[[ 'c1', 'c2', 'c3', 'w1', 'w2', 'w3', 'd1', 'd2', 'd3']]


j1 = sm.OLS(v1, X3).fit()
j2 = sm.OLS(v2, X3).fit()
j3 = sm.OLS(v3, X3).fit()
j4 = sm.OLS(v4, X3).fit()

k1 = sm.OLS(v1, X4).fit()
k2 = sm.OLS(v2, X4).fit()
k3 = sm.OLS(v3, X4).fit()
k4 = sm.OLS(v4, X4).fit()

l1 = sm.OLS(v1, X5).fit()
l2 = sm.OLS(v2, X5).fit()
l3 = sm.OLS(v3, X5).fit()
l4 = sm.OLS(v4, X5).fit()

#z1 = sm.OLS(v1, X6).fit()

#j1.summary()
#j2.summary()
#j3.summary()
#j4.summary()

#k1.summary()
#k2.summary()
#k3.summary()
#k4.summary()

#l1.summary()
#l2.summary()
#l3.summary()
l4.summary()

#z1.summary()

/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1263: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:978: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     GA   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     4.459
Date:                Wed, 25 Jul 2018   Prob (F-statistic):           0.000515
Time:                        09:41:54   Log-Likelihood:                -1605.7
No. Observations:                 828   AIC:                             3223.
Df Residuals:                     822   BIC:                             3252.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
d0             3.3086      0.188     17.634      0.000         2.940     3.677
d1             3.2323      0.120     26.935      0.000         2.997     3.468
d2             2.8199      0.105     26.979      0.000         2.615     3.025
d3             2.7095      0.126     21.467      0.000         2.462     2.957
d4             2.4490      0.171     14.357      0.000         2.114     2.784
d5             2.8182      0.509      5.535      0.000         1.819     3.818
d6                  0          0        nan        nan             0         0
==============================================================================
Omnibus:                       14.822   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.681
Skew:                           0.298   Prob(JB):                     0.000649
Kurtosis:                       2.736   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""